# PyTranscribe

> Convert audio files into test output using AWS Transcribe

In [ ]:
#| default_exp core

In [ ]:
#| hide
from nbdev.showdoc import *

In [ ]:
#| export
from botocore.exceptions import ClientError
from pathlib import Path
from datetime import datetime
import json
import boto3
import logging
import os
import filetype
import time

class JsonToDictation:
    "A class that will convert the AWS Transcribe to Dictation"

    def __init__(self,
                 speakers,
                 data # JSON data that has been generated by AWS Transcribe
                ):
        self.transcribe_data = data
        self.titles = []
        self.num_speakers = 0
        self.speakers = speakers
        self.speaker_map = set()
        self.text = ''
        self.load()


    def load(self):
        self.titles = self.transcribe_data["results"]["transcripts"]
        self.num_speakers = self.transcribe_data["results"]["speaker_labels"]["speakers"] 
        speaker_set = set(self.speakers)
        #self.num_speakers = len(speaker_set)
        self.getSpeakerMap()
        
    def getSpeakerMap(self):
        for segment in self.transcribe_data["results"]["speaker_labels"]["segments"]:
            speaker = segment["speaker_label"]            
            self.speaker_map.add(speaker)
        #print("SPEAKER_MAP: "+str(self.speaker_map))
        return (self.speaker_map)

    def applyFormattingRules(self, word: str) -> str:
        # translate some spellings into standarized format
        if word.upper() == "OK":
            return "Okay "
        if word == "Cuz":
            return "Because "
        if word == "cuz":
            return "because "
            
        # remove um and ah from text output
        if word.upper() == "UM":
            return ""
        if word.upper() == "AH":
            return ""

        # none of the above, then return word with a space after
        return f"{word} "       
        
    def convert(self):
        str = ''
        current_speaker = ''
        for item in self.transcribe_data["results"]["items"]:
            alternatives = item["alternatives"]
            speaker_label = item["speaker_label"]
            #print(f"{speaker_label=}")
            speaker_list = list(self.speaker_map)
            speaker_index = speaker_list.index(speaker_label)
            #print(f"{speaker_index=}")
            speaker = self.speakers[speaker_index]
            #print(f"{speaker=}")
            word = alternatives[0]["content"]
            type = item["type"]

            if current_speaker != speaker:
                if current_speaker != '':
                    str += "\n"
                    
                current_speaker = speaker                
                str += f"{speaker}:\t"

            if type == "punctuation":
                str = str[:-1]

            str += self.applyFormattingRules(word)
            
        self.text = str        
        return (str)        
            
    def summary(self):
        print(f"{self.num_speakers=}")
        print(f"{self.speakers=}")
        print(f"{self.speaker_map=}")
        print(f"{speakers=}")


In [ ]:
#| hide
import nbdev; nbdev.nbdev_export()


In [ ]:
with open('test_data/transcribe_example.json') as transcribe_file:
  transcribe_json = json.load(transcribe_file)

speakers = ("BOB","JANE")
dictation = JsonToDictation(speakers,transcribe_json)
dictation.summary()

assert(dictation.num_speakers==2)

self.num_speakers=2
self.speakers=('BOB', 'JANE')
self.speaker_map={'spk_0', 'spk_1'}
speakers=('BOB', 'JANE')


In [ ]:
text = dictation.convert()
print (text)
assert(text=="BOB:\tbeen on hold for an hour. \nJANE:\tSorry about that. ")

BOB:	been on hold for an hour. 
JANE:	Sorry about that. 


In [ ]:
#| export

class AWSTranscribe:
    "A class that will upload audio files to s3 and then execute transcription jobs for them using AWSTranscribe"

    def __init__(self, input_bucket, output_bucket, num_speakers, input_file_name):
        self.s3_client = boto3.client('s3')
        self.transcribe_client = boto3.client('transcribe')
        self.input_bucket = input_bucket
        self.output_bucket = output_bucket
        self.num_speakers = num_speakers
        self.file_name = input_file_name
        self.ouput_file = ""
        self.out_uri = ""
        self.out_key = ""
        

    def printInputBucketContents(self):
        return self.printBucketContents(self.input_bucket)

    def printOutputBucketContents(self):
        return self.printBucketContents(self.output_bucket)
    
        
    def printBucketContents(self, bucket):
        response = self.s3_client.list_objects_v2(Bucket=bucket)
        if 'Contents' not in response:
            print("Empty")
            return
            
        for content in response['Contents']:
            obj_dict = self.s3_client.get_object(Bucket=bucket, Key=content['Key'])
            print(content['Key'], obj_dict['LastModified'])

    def uploadFile(self):
        file_name = self.file_name
        object_name = os.path.basename(file_name)
        try:
            response = self.s3_client.upload_file(file_name, self.input_bucket, object_name)
        except ClientError as e:
            print(error(e))
            logging.exception("Cannot upload file to s3: "+e)
            raise

        try:
            waiter = self.s3_client.get_waiter('object_exists')
            waiter.wait(Bucket=self.input_bucket, Key = object_name, WaiterConfig={'Delay': 20, 'MaxAttempts': 50})
            #print('Object exists: ' + self.input_bucket +'/'+object_name)
        except ClientError as e:
            raise Exception( "boto3 client error in use_waiters_check_object_exists: " + e.__str__())
        except Exception as e:
            raise Exception( "Unexpected error in use_waiters_check_object_exists: " + e.__str__())

        return object_name

    def getFileType(self):
        file_name = self.file_name
        kind = filetype.guess(file_name)
        if kind is None:
            print('Cannot guess file type!')
            logger.exception("Cannot guess file type!")
            raise

        #print('File extension: %s' % kind.extension)
        #print('File MIME type: %s' % kind.mime)        
        return kind.extension

    def createTranscriptionJob(self):
        file_name = self.file_name
        media_format = self.getFileType()
        #print(media_format)

        # upload file to s3 (inputbucket)
        key = self.uploadFile()
        
        # construct media_url
        media_uri = f"s3://{self.input_bucket}/{key}"
        #print (f"{media_uri=}")
        
        # construct job_name
        now = datetime.now()
        job_name = now.strftime("%Y-%m-%d") + "--" + Path(file_name).stem        
        

        # constuct output file namne
        out_key = f"{job_name}.json"

        # construct output uri
        out_uri = f"s3://{self.output_bucket}/{out_key}"

        # check if the job already exists
        if not self.jobExists(job_name):
                
            # create transcription job    
            print(f"Creating transcriptions job: {job_name} for media file: {media_uri}...")
            try:
                job_args = {
                    "TranscriptionJobName": job_name,
                    "Media": {"MediaFileUri": media_uri},
                    "MediaFormat": media_format,
                    "LanguageCode": os.environ['DEFAULT_LANGUAGE_CODE'],
                    "OutputBucketName": self.output_bucket,
                    "OutputKey": out_key
                }
                    
                job_args["Settings"] = {        
                    'ShowSpeakerLabels': True,
                    'MaxSpeakerLabels': self.num_speakers
                }
            
                response = self.transcribe_client.start_transcription_job(**job_args)
                job = response["TranscriptionJob"]
                print ("Started transcription job %s.", job_name)
            except ClientError:
                logger.exception("Couldn't start transcription job %s.", job_name)
                raise


        # wait for job to complete
        # TODO: turn this into a waiter (like in upload)
        print ("Waiting for job to complete...")
        while not self.isJobComplete(job_name):
            time.sleep(int(os.environ['JOB_SLEEP_INTERVAL']))

        self.out_uri = out_uri
        self.out_key = out_key
        print (f"Sucess.  File saved at {out_uri}")


    def downloadTranscription(self):
        self.output_filename =  self.out_key

        print(f"Download json transcription file from AWS ({self.out_uri} to {self.out_key} ...")
        #with open(self.output_filename, 'wb') as f:
        #    self.s3_client.download_fileobj(self.output_bucket, self.out_key, f)
        self.s3_client.download_file(self.output_bucket, self.output_filename, self.out_key)

        print ("Complete")
        return self.output_filename

        
    def getTranscriptionJob(self, job_name):
        try:
            response = self.transcribe_client.get_transcription_job(TranscriptionJobName=job_name)
        except:
            return None
        return response

    def isJobComplete(self, job_name) -> bool:        
        result = self.getTranscriptionJob(job_name)
        return str(result['TranscriptionJob']['TranscriptionJobStatus']) == 'COMPLETED'

    def jobExists(self, job_name) -> bool:
        result = self.getTranscriptionJob(job_name)
        if result is None:
            return False
        return 'TranscriptionJob' in result

In [ ]:
input_file_name = '../2023_02_27_11_49_35_CH12_MIX.mp3'
awstrans = AWSTranscribe(os.environ['INPUT_BUCKET'],os.environ['OUTPUT_BUCKET'], 2, input_file_name)
#awstrans.uploadFile(file_name)
awstrans.printInputBucketContents()

2023_02_27_11_49_35_CH12_MIX.mp3 2024-09-08 04:37:23+00:00


In [ ]:
awstrans.createTranscriptionJob()

Waiting for job to complete...
Sucess.  File saved at s3://weinberg-dictations/2024-09-08--2023_02_27_11_49_35_CH12_MIX.json


In [ ]:
awstrans.downloadTranscription()

Download json transcription file from AWS (s3://weinberg-dictations/2024-09-08--2023_02_27_11_49_35_CH12_MIX.json to 2024-09-08--2023_02_27_11_49_35_CH12_MIX.json ...
Complete


'2024-09-08--2023_02_27_11_49_35_CH12_MIX.json'